In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('/kaggle/input/market-basket-analysis/Assignment-1_Data.xlsx')
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   BillNo      522064 non-null  object        
 1   Itemname    520609 non-null  object        
 2   Quantity    522064 non-null  int64         
 3   Date        522064 non-null  datetime64[ns]
 4   Price       522064 non-null  float64       
 5   CustomerID  388023 non-null  float64       
 6   Country     522064 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 27.9+ MB


In [4]:
df.isnull().sum()

BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64

In [5]:
df.dropna(subset=["Itemname"],inplace=True)
df = df[df["Quantity"]>0]
df['CustomerID'].fillna(method='ffill', inplace=True)
df.isnull().sum()

BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
Country       0
dtype: int64

In [6]:
df['Country'].value_counts()

Country
United Kingdom          485694
Germany                   9042
France                    8408
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Sweden                     451
Unspecified                446
Austria                    398
Poland                     330
Japan                      321
Israel                     295
Hong Kong                  284
Singapore                  222
Iceland                    182
USA                        179
Greece                     145
Malta                      112
United Arab Emirates        68
RSA                         58
Lebanon                     45
Lithuania                   35
Brazil                      32
Bahrain                     18
Saudi Arabia                 9
Name: count, dtype: int64

In [7]:
df = df[df['Country']=='United Kingdom']
df.shape

(485694, 7)

In [8]:
df['Itemname'] = df['Itemname'].str.strip()
df['BillNo'] = df['BillNo'].astype('str')

In [9]:
basket = (df.groupby(['BillNo','Itemname'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('BillNo')
          )
basket

Itemname,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A563185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1 :
        return 1

In [11]:
basket_set = basket.applymap(encode_units)
frequent_items = apriori(basket_set , min_support= 0.02 , use_colnames= True)
frequent_items

,support,itemsets
0,0.024019,(3 STRIPEY MICE FELTCRAFT)
1,0.021655,(4 TRADITIONAL SPINNING TOPS)
2,0.046994,(6 RIBBONS RUSTIC CHARM)
3,0.020391,(60 CAKE CASES DOLLY GIRL DESIGN)
4,0.031934,(60 CAKE CASES VINTAGE CHRISTMAS)
...,...,...
388,0.020226,"(WOODEN STAR CHRISTMAS SCANDINAVIAN, WOODEN HE..."
389,0.027097,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY..."
390,0.020117,"(JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKA..."
391,0.022260,"(JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKA..."


In [12]:
# basket.iloc[:, 1:] = basket.iloc[:, 1:].astype(bool)
# frequent_itemsets = apriori(basket.iloc[:, 1:], min_support=0.01, use_colnames=True)
# frequent_itemsets

In [13]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETROSPOT CAKE CASES),0.041387,0.062053,0.022480,0.543161,8.753114,0.019912,2.053121,0.923997
1,(PACK OF 72 RETROSPOT CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.062053,0.041387,0.022480,0.362267,8.753114,0.019912,1.503158,0.944355
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.051116,0.048148,0.030944,0.605376,12.573307,0.028483,2.412051,0.970051
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.048148,0.051116,0.030944,0.642694,12.573307,0.028483,2.655663,0.967027
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.051116,0.036056,0.021601,0.422581,11.720171,0.019758,1.669400,0.963950
...,...,...,...,...,...,...,...,...,...,...
203,"(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)",(JUMBO SHOPPER VINTAGE RED PAISLEY),0.038364,0.062273,0.021106,0.550143,8.834339,0.018717,2.084501,0.922184
204,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO STOR...",(JUMBO BAG RED RETROSPOT),0.028196,0.106519,0.021106,0.748538,7.027296,0.018102,3.553147,0.882583
205,(JUMBO BAG RED RETROSPOT),"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO STOR...",0.106519,0.028196,0.021106,0.198142,7.027296,0.018102,1.211941,0.959950
206,(JUMBO SHOPPER VINTAGE RED PAISLEY),"(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)",0.062273,0.038364,0.021106,0.338923,8.834339,0.018717,1.454651,0.945697


In [14]:
best_rules = rules[(rules['lift'] >= 0.5) &
      (rules['confidence'] >= 0.5)]

In [15]:
best_rules.sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
186,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.030010,0.051336,0.027097,0.902930,17.588775,0.025556,9.773033,0.972325
184,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.031714,0.052600,0.027097,0.854419,16.243790,0.025429,6.507737,0.969174
27,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.038639,0.051336,0.031714,0.820768,15.988282,0.029730,5.292945,0.975132
192,"(JUMBO BAG PINK POLKADOT, JUMBO SHOPPER VINTAG...",(JUMBO BAG RED RETROSPOT),0.024953,0.106519,0.020117,0.806167,7.568323,0.017459,4.609552,0.890081
198,"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SUKI)",(JUMBO BAG RED RETROSPOT),0.027756,0.106519,0.022260,0.801980,7.529013,0.019304,4.512081,0.891937
...,...,...,...,...,...,...,...,...,...,...
189,(ROSES REGENCY TEACUP AND SAUCER),"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",0.052600,0.031714,0.027097,0.515152,16.243790,0.025429,1.997090,0.990540
15,(CHARLOTTE BAG SUKI DESIGN),(WOODLAND CHARLOTTE BAG),0.044190,0.037540,0.022700,0.513682,13.683635,0.021041,1.979074,0.969775
97,(JUMBO BAG WOODLAND ANIMALS),(JUMBO SHOPPER VINTAGE RED PAISLEY),0.040398,0.062273,0.020556,0.508844,8.171138,0.018041,1.909222,0.914565
168,(RED RETROSPOT CHARLOTTE BAG),(STRAWBERRY CHARLOTTE BAG),0.049742,0.036990,0.025173,0.506077,13.681384,0.023333,1.949718,0.975427
